In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [ ]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [4]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df= np.array(df)#convert the data frame into numpy array
df.shape

(42000, 785)

In [6]:
m, n = df.shape
np.random.shuffle(df)#to avoid overfitting or under fitting 
                           #this make us to take data colum wise rather than row wise
data_dev = df[0:1000].T
y_test = data_dev[0]#lobels stored 
x_test = data_dev[1:n]# pixels stored 
x_test = x_test / 255#normilizing the data

data_train = df[1000:m].T
y_train = data_train[0]
x_train = data_train[1:n]
x_train = x_train / 255


In [7]:
def params_inti():
    w1=np.random.rand(41000,784)-0.5#intilizing waiegts to all elemts in the array 
    b1=np.random.rand(10,1)-0.5
    w2=np.random.rand(10,256)-0.5
    b2=np.random.rand(10,1)-0.5
    return w1,b1,w2,b2

def ReLU(z):# defining relu and soft max activasion functions 
    return np.maximum(z, 0)

def DerReLU(z):
    return z>0


def softmax(z):
    a = np.exp(z) / sum(np.exp(z))
    return a

def Forwardprob(w1,b1,w2,b2,x):
    z1=w1.dot(x)+b1
    a1=ReLU(z1)
    z2=w2.dot(a1)+b2
    a2=softmax(z2)
    return z1,a1,z2,a2

def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max() + 1))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y

def backwardprop(z1,a1,z2,a2,w1,w2,x,y):
    one_hot_y=one_hot(y)
    dz2=a2-one_hot_y
    dw2=1/m*dz2.dot(a1.T)
    db2=1/m*np.sum(dz2)
    dz1=w2.T.dot(dz2)*DerReLU(z1)
    dw1=1/m*dz1.dot(x.T)
    db1=1/m*np.sum(dz1)
    return dw1,db1,dw2,db2
def update_parms(w1,b1,w2,b2,dw1,db1,dw2,db2,lr):
    w1=w1-lr*dw1
    b1=b1-lr*db1
    w2=w2-lr*dw2
    b2=b2-lr*db2
    return w1,b1,w2,b2

    



In [8]:
def get_predictions(a2):
    return np.argmax(a2, 0)



def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

def gradient_descent(x, y, lr, epochs):
    w1, b1, w2, b2 = params_inti()
    for i in range(epochs):
        z1, a1, z2, a2 = Forwardprob(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = backwardprop(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_parms(w1, b1, w2, b2, dw1, db1, dw2, db2, lr)
        if i % 10 == 0:
            print("epoch: ", i)
            predictions = get_predictions(a2)
            acuracy=get_accuracy(predictions, y)
            print(get_accuracy(predictions, y))
    return w1, b1, w2, b2,acuracy



In [ ]:
w1, b1, w2, b2 = gradient_descent(x_train,y_train,0.01,500)

predictions

In [ ]:
from matplotlib import pyplot as plt

def make_predictions(x, w1, b1, w2, b2):
    _, _, _, a2 = Forwardprob(w1, b1, w2, b2, x)
    predictions = get_predictions(a2)
    return predictions

def test_prediction(index, w1, b1, w2, b2):
    current_image = x_train[:, index, None]
    prediction = make_predictions(x_train[:, index, None], w1, b1, w2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
for i in range(20):#making 10 predictins
    test_prediction(i, w1, b1, w2, b2)
    